In [2]:
# Install the required packages
!pip install scikit-fuzzy

import os
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from ipywidgets import widgets, HBox, VBox, Dropdown, FloatSlider, Button, Output, Text
from IPython.display import display, clear_output


  Using cached scikit-fuzzy-0.4.2.tar.gz (993 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894078 sha256=f5da7cc999d0196d8337bda7470ca86c6e8b4957cf387b23e491075b37d788ae
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [4]:

# Define membership functions, antecedents, and consequents
def define_membership_function(universe, points):
    return fuzz.trimf(universe, points)

def create_antecedent(name, universe, points_dict):
    antecedent = ctrl.Antecedent(universe, name)
    for label, points in points_dict.items():
        antecedent[label] = define_membership_function(universe, points)
    return antecedent

def create_consequent(name, universe, points_dict):
    consequent = ctrl.Consequent(universe, name)
    for label, points in points_dict.items():
        consequent[label] = define_membership_function(universe, points)
    return consequent

# Define universes and membership points for each symptom and diagnosis
fever_universe = np.arange(36, 41.1, 0.1)
fever_points = {'Low': [36, 37, 38], 'Medium': [37.5, 38.5, 39.5], 'High': [39, 40, 41]}
fever = create_antecedent('fever', fever_universe, fever_points)

headache_universe = np.arange(0, 10.1, 1)
headache_points = {'Low': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
headache = create_antecedent('headache', headache_universe, headache_points)

cough_universe = np.arange(0, 10.1, 1)
cough_points = {'Mild': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
cough = create_antecedent('cough', cough_universe, cough_points)

fatigue_universe = np.arange(0, 10.1, 1)
fatigue_points = {'Low': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
fatigue = create_antecedent('fatigue', fatigue_universe, fatigue_points)

sore_throat_universe = np.arange(0, 10.1, 1)
sore_throat_points = {'Mild': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
sore_throat = create_antecedent('sore_throat', sore_throat_universe, sore_throat_points)

shortness_of_breath_universe = np.arange(0, 10.1, 1)
shortness_of_breath_points = {'Mild': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
shortness_of_breath = create_antecedent('shortness_of_breath', shortness_of_breath_universe, shortness_of_breath_points)

nausea_universe = np.arange(0, 10.1, 1)
nausea_points = {'Mild': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
nausea = create_antecedent('nausea', nausea_universe, nausea_points)

diagnosis_universe = np.arange(0, 10.1, 1)
diagnosis_points = {'Unknown': [0, 1, 2], 'Respiratory_allergy': [1, 2, 3], 'Common_cold': [2, 3, 4],
                    'Mild_viral_infection': [3, 4, 5], 'Sinusitis': [4, 5, 6], 'Flu': [5, 6, 7],
                    'Bronchitis': [6, 7, 8], 'Bacterial_infection': [7, 8, 9], 'Pneumonia': [8, 9, 10]}
diagnosis = create_consequent('diagnosis', diagnosis_universe, diagnosis_points)

# Define rules mapping input patterns to diagnoses
rules = [
    ctrl.Rule(antecedent=(fever['Low'] & headache['Low'] & cough['Mild']), consequent=diagnosis['Respiratory_allergy']),
    ctrl.Rule(antecedent=(fever['Low'] & headache['Low'] & cough['Moderate']), consequent=diagnosis['Common_cold']),
    ctrl.Rule(antecedent=(fever['Low'] & headache['Moderate'] & cough['Mild']), consequent=diagnosis['Common_cold']),
    ctrl.Rule(antecedent=(fever['Low'] & headache['Moderate'] & cough['Moderate']), consequent=diagnosis['Common_cold']),
    ctrl.Rule(antecedent=(fever['Medium'] & headache['Moderate'] & cough['Moderate']), consequent=diagnosis['Mild_viral_infection']),
    ctrl.Rule(antecedent=(fever['Medium'] & headache['Severe'] & cough['Mild']), consequent=diagnosis['Mild_viral_infection']),
    ctrl.Rule(antecedent=(fever['Low'] & headache['Severe'] & cough['Mild']), consequent=diagnosis['Sinusitis']),
    ctrl.Rule(antecedent=(fever['Medium'] & headache['Moderate'] & cough['Severe']), consequent=diagnosis['Flu']),
    ctrl.Rule(antecedent=(fever['Medium'] & fatigue['Severe'] & shortness_of_breath['Moderate']), consequent=diagnosis['Flu']),
    ctrl.Rule(antecedent=(fever['Medium'] & nausea['Moderate'] & headache['Moderate']), consequent=diagnosis['Flu']),
    ctrl.Rule(antecedent=(fever['High'] & headache['Moderate'] & cough['Severe']), consequent=diagnosis['Bronchitis']),
    ctrl.Rule(antecedent=(fever['High'] & headache['Severe'] & cough['Severe']), consequent=diagnosis['Pneumonia']),
    ctrl.Rule(antecedent=(fever['High'] & shortness_of_breath['Severe'] & fatigue['Severe']), consequent=diagnosis['Pneumonia']),
    ctrl.Rule(antecedent=(fever['High'] & sore_throat['Severe'] & headache['Severe']), consequent=diagnosis['Bacterial_infection']),
]

# Create the control system with the defined rules
diagnostic_system = ctrl.ControlSystem(rules)
simulation_diagnostic = ctrl.ControlSystemSimulation(diagnostic_system)

# Define symptom and diagnosis options
symptom_options = ['Fever', 'Headache', 'Cough', 'Fatigue', 'Sore Throat', 'Shortness of Breath', 'Nausea']
diagnosis_labels = ['Unknown', 'Respiratory_allergy', 'Common_cold', 'Mild_viral_infection', 'Sinusitis', 'Flu', 'Bronchitis', 'Bacterial_infection', 'Pneumonia']

# Define interactive widgets with styling
symptom_dropdown = Dropdown(options=symptom_options, description='Symptom:', style={'description_width': 'initial', 'width': '300px'})
symptom_value = FloatSlider(value=37.0, min=36.0, max=41.0, step=0.1, description='Value:', style={'description_width': 'initial', 'width': '300px'})
btn_add_symptom = Button(description="Add Symptom", button_style='success', layout={'width': '150px'})
btn_remove_symptom = Button(description="Remove Symptom", button_style='danger', layout={'width': '150px'})
btn_diagnose = Button(description="Diagnose", button_style='info', layout={'width': '150px'})
output_diagnosis = Output()
login_text = Text(description='Username:', style={'description_width': 'initial', 'width': '300px'})
password_text = Text(description='Password:', style={'description_width': 'initial', 'width': '300px'}, layout=widgets.Layout(width='300px'))
btn_login = Button(description='Login', button_style='primary', layout=widgets.Layout(width='150px'))
btn_logout = Button(description='Logout', button_style='warning', layout=widgets.Layout(width='150px'))
btn_register = Button(description='Register', button_style='success', layout=widgets.Layout(width='150px'))
btn_symptom_trends = Button(description='View Symptom Trends', button_style='info', layout=widgets.Layout(width='200px'))
btn_diagnosis_distribution = Button(description='View Diagnosis Distribution', button_style='info', layout=widgets.Layout(width='200px'))

# Initialize database connection
db_path = 'health_diagnostic_system.db'
conn = sqlite3.connect(db_path)
c = conn.cursor()

# Create tables if they do not exist
c.execute('''CREATE TABLE IF NOT EXISTS users (
             id INTEGER PRIMARY KEY AUTOINCREMENT,
             username TEXT NOT NULL,
             password TEXT NOT NULL)''')

c.execute('''CREATE TABLE IF NOT EXISTS diagnoses (
             id INTEGER PRIMARY KEY AUTOINCREMENT,
             user_id INTEGER,
             symptoms TEXT,
             diagnosis TEXT,
             FOREIGN KEY(user_id) REFERENCES users(id))''')

conn.commit()

# Define list to store selected symptoms and user information
selected_symptoms = []
user = None
diagnostic_history = {}

def update_symptom_value_range(change):
    symptom = change['new']
    if symptom == 'Fever':
        symptom_value.min = 36.0
        symptom_value.max = 41.0
        symptom_value.step = 0.1
        symptom_value.value = 37.0
    else:
        symptom_value.min = 0.0
        symptom_value.max = 10.0
        symptom_value.step = 1.0
        symptom_value.value = 5.0

def add_symptom(b):
    if not user:
        display_message("Please log in first.")
        return
    symptom = symptom_dropdown.value
    value = symptom_value.value
    if symptom not in [s[0] for s in selected_symptoms]:
        selected_symptoms.append((symptom, value))
        display_selected_symptoms()
    else:
        display_message(f"Symptom '{symptom}' already added.")

def remove_symptom(b):
    if not user:
        display_message("Please log in first.")
        return
    if selected_symptoms:
        removed_symptom = selected_symptoms.pop()
        display_message(f"Symptom removed: {removed_symptom}")
        display_selected_symptoms()

def diagnose(b):
    if not user:
        display_message("Please log in first.")
        return
    if selected_symptoms:
        display_selected_symptoms()
        # Set default values for all antecedents
        defaults = {
            'fever': 36.0,
            'headache': 0.0,
            'cough': 0.0,
            'fatigue': 0.0,
            'sore_throat': 0.0,
            'shortness_of_breath': 0.0,
            'nausea': 0.0
        }
        for symptom, value in selected_symptoms:
            key = symptom.lower().replace(" ", "_")
            defaults[key] = value
        for key, value in defaults.items():
            simulation_diagnostic.input[key] = value
        simulation_diagnostic.compute()
        diagnosis_result = diagnosis_labels[int(round(simulation_diagnostic.output['diagnosis']))]
        display_message(f"Diagnosis result: {diagnosis_result}")
        save_diagnostic_result(diagnosis_result)
    else:
        display_message("No symptoms selected.")

def login(b):
    global user
    username = login_text.value
    password = password_text.value
    c.execute("SELECT * FROM users WHERE username=? AND password=?", (username, password))
    result = c.fetchone()
    if result:
        user = username
        display_message(f"Logged in as: {user}")
        display(logout_button, HBox([symptom_dropdown, symptom_value]), HBox([btn_add_symptom, btn_remove_symptom]), btn_diagnose, btn_symptom_trends, btn_diagnosis_distribution)
    else:
        display_message("Incorrect username or password.")

def logout(b):
    global user
    user = None
    selected_symptoms.clear()
    display_message("Logged out.")
    display_login()

def register(b):
    username = login_text.value
    password = password_text.value
    c.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, password))
    conn.commit()
    display_message(f"User {username} registered successfully.")

def plot_symptom_trends(username):
    c.execute("SELECT symptoms FROM diagnoses WHERE user_id=(SELECT id FROM users WHERE username=?)", (username,))
    data = c.fetchall()
    if not data:
        display_message("No symptom data available for this user.")
        return

    symptom_records = [eval(record[0]) for record in data]
    df = pd.DataFrame(symptom_records)
    df.plot(marker='o')
    plt.title(f"Symptom Trends for {username}")
    plt.xlabel("Record")
    plt.ylabel("Severity")
    plt.grid(True)
    plt.show()

def plot_diagnosis_distribution():
    c.execute("SELECT diagnosis FROM diagnoses")
    data = c.fetchall()
    if not data:
        display_message("No diagnosis data available.")
        return

    diagnoses = [record[0] for record in data]
    df = pd.DataFrame(diagnoses, columns=["Diagnosis"])
    df_counts = df['Diagnosis'].value_counts()
    df_counts.plot(kind='pie', autopct='%1.1f%%')
    plt.title("Diagnosis Distribution")
    plt.ylabel('')
    plt.show()

def display_message(message):
    with output_diagnosis:
        clear_output(wait=True)
        print(message)

def display_selected_symptoms():
    with output_diagnosis:
        clear_output(wait=True)
        print("Selected Symptoms:")
        for s, v in selected_symptoms:
            print(f"{s}: {v}°C" if s == "Fever" else f"{s}: {v}")

def save_diagnostic_result(result):
    user_id_query = "SELECT id FROM users WHERE username=?"
    c.execute(user_id_query, (user,))
    user_id = c.fetchone()[0]
    symptoms_str = str(selected_symptoms)
    insert_query = "INSERT INTO diagnoses (user_id, symptoms, diagnosis) VALUES (?, ?, ?)"
    c.execute(insert_query, (user_id, symptoms_str, result))
    conn.commit()
    display_message("Diagnostic result saved.")

# Bind event handlers to buttons
btn_add_symptom.on_click(add_symptom)
btn_remove_symptom.on_click(remove_symptom)
btn_diagnose.on_click(diagnose)
btn_login.on_click(login)
btn_logout.on_click(logout)
btn_register.on_click(register)
btn_symptom_trends.on_click(lambda _: plot_symptom_trends(user))
btn_diagnosis_distribution.on_click(plot_diagnosis_distribution)

def display_login():
    display(VBox([login_text, password_text, HBox([btn_login, btn_register]), output_diagnosis]))

display_login()
